In [ ]:
from local_binary_pattern import FeatureExtractor
from preprocessing import *
from sklearn.svm import SVC
from imutils import paths
import time
import cv2
import os
from matplotlib import pyplot as plt
from natsort import natsorted
from sklearn.neighbors import KNeighborsClassifier

train_line_voting = False
predict_line_voting = True

def get_features(root_dir, file, feature_extractor):
    labels = []
    features = []
    for writer in os.listdir(os.path.join(root_dir, file)):
        print(writer)
        for sample in paths.list_images(os.path.join(root_dir, file, writer)):
            print(sample)
            gray = cv2.imread(sample, 0)
            cropped_img = Preprocessor.paragraph_extraction(gray)
            line_boundries = Preprocessor.line_segmentation(cropped_img)
            hist = None
            for line in line_boundries:
                lbp = list(
                    feature_extractor.local_binary_pattern(cropped_img[line[0]:line[2], line[1]:line[3]]).ravel())
                if train_line_voting:
                    hist = feature_extractor.histogram(lbp)
                    labels.append(str(writer))
                    features.append(hist)
#                     plt.plot(hist)
#                     plt.show()
                else:
                    hist = feature_extractor.histogram_acc(lbp, hist)
            if not train_line_voting:
                hist /= (hist.mean())
                labels.append(str(writer))
                features.append(hist)
            
#                 plt.plot(hist)
#                 plt.show()
            
#             Preprocessor.draw_segmented_lines(cropped_img, line_boundries)
#             Preprocessor.display_image(cropped_img)
    return features, labels


def get_prediction(root_dir, file, feature_extractor, model):
    gray = cv2.imread(os.path.join(root_dir, file, 'test.png'), 0)
    cropped_img = Preprocessor.paragraph_extraction(gray)
    line_boundries = Preprocessor.line_segmentation(cropped_img)
    i = 0
    lst = []
    hist = None
    for line in line_boundries:
        if line[2] > line[0] and line[3] > line[1]:
            lbp = list(feature_extractor.local_binary_pattern(cropped_img[line[0]:line[2], line[1]:line[3]]).ravel())
            if predict_line_voting:
                hist = feature_extractor.histogram(lbp)
                lst.append(model.predict(hist.reshape(1, -1)))
                print("line " + str(i) + " presidctions is: ")
                print(lst[i])
            else:
                hist = feature_extractor.histogram_acc(lbp, hist)
            i += 1
    if not predict_line_voting:
        hist /= (hist.mean())
        return model.predict(hist.reshape(1, -1))
    else:
        return max(lst, key=lst.count)


def main():
    root_dir = 'testCases'
    time_file = open("time.txt", "w")
    results_file = open("results.txt", "w")
    actual_result=open("actual_results.txt", "r")
    actual_result_lines=actual_result.readlines()
    feature_extractor = FeatureExtractor(24, 8)
    correct_classification = 0
    tests = natsorted(os.listdir(root_dir))
    num_of_tests = len(tests)
    for test in tests:
        print("Test: ", test)
        start_time = time.time()
        features, labels = get_features(root_dir, test, feature_extractor)

#         model = KNeighborsClassifier(n_neighbors=3)
        model = SVC(C=0.5, gamma='auto', probability=True)
        model.fit(features, labels)

        prediction = get_prediction(root_dir, test, feature_extractor, model)[0]
        time_taken = round(time.time() - start_time, 2)
        if int(prediction)==int(actual_result_lines[int(test)-1]):
            correct_classification += 1
        else:
            print("Wrong classified at TestCase ",test)
        print("Prediction: " + prediction)
        print("Time taken: " + str(time_taken))
        results_file.write(prediction + '\n')
        time_file.write(str(time_taken) + '\n')
    results_file.close()
    time_file.close()

    acc = ( correct_classification / num_of_tests ) * 100.0
    print("Accuracy is : "+str(acc)+' % ')


if __name__ == '__main__':
    main()

Test:  1
272
testCases\1\272\f04-004.png
testCases\1\272\f04-011.png
335
testCases\1\335\g06-018d.png
testCases\1\335\g06-026d.png
346
testCases\1\346\g06-011o.png
testCases\1\346\g06-018o.png
test.png
line 0 presidctions is: 
['346']
line 1 presidctions is: 
['346']
line 2 presidctions is: 
['346']
line 3 presidctions is: 
['346']
line 4 presidctions is: 
['346']
line 5 presidctions is: 
['346']
line 6 presidctions is: 
['346']
line 7 presidctions is: 
['346']
Prediction: 346
Time taken: 15.0
Test:  2
193
testCases\2\193\d04-081.png
testCases\2\193\f04-093.png
25
testCases\2\25\a02-017.png
testCases\2\25\a02-020.png
88
testCases\2\88\b01-014.png
testCases\2\88\b01-018.png
test.png
line 0 presidctions is: 
['193']
line 1 presidctions is: 
['193']
line 2 presidctions is: 
['88']
line 3 presidctions is: 
['193']
line 4 presidctions is: 
['88']
line 5 presidctions is: 
['193']
Wrong classified at TestCase  2
Prediction: 193
Time taken: 13.77
Test:  3
213
testCases\3\213\e01-050.png
testCa

351
testCases\17\351\g07-014.png
testCases\17\351\g07-018a.png
61
testCases\17\61\a04-047.png
testCases\17\61\a04-050.png
test.png
line 0 presidctions is: 
['351']
line 1 presidctions is: 
['351']
line 2 presidctions is: 
['351']
line 3 presidctions is: 
['351']
line 4 presidctions is: 
['351']
line 5 presidctions is: 
['351']
line 6 presidctions is: 
['351']
line 7 presidctions is: 
['351']
line 8 presidctions is: 
['351']
Prediction: 351
Time taken: 15.99
Test:  18
354
testCases\18\354\g07-065.png
testCases\18\354\g07-069a.png
551
testCases\18\551\m06-019.png
testCases\18\551\n06-148.png
555
testCases\18\555\n01-020.png
testCases\18\555\n01-031.png
test.png
line 0 presidctions is: 
['555']
line 1 presidctions is: 
['555']
line 2 presidctions is: 
['555']
line 3 presidctions is: 
['555']
line 4 presidctions is: 
['555']
line 5 presidctions is: 
['555']
line 6 presidctions is: 
['555']
line 7 presidctions is: 
['555']
line 8 presidctions is: 
['555']
line 9 presidctions is: 
['555']
Pr

line 2 presidctions is: 
['209']
line 3 presidctions is: 
['209']
line 4 presidctions is: 
['209']
line 5 presidctions is: 
['209']
line 6 presidctions is: 
['209']
line 7 presidctions is: 
['209']
line 8 presidctions is: 
['209']
line 9 presidctions is: 
['209']
line 10 presidctions is: 
['209']
Prediction: 209
Time taken: 13.67
Test:  33
37
testCases\33\37\a02-102.png
testCases\33\37\a02-106.png
385
testCases\33\385\h07-013.png
testCases\33\385\h07-017.png
63
testCases\33\63\a04-081.png
testCases\33\63\a04-085.png
test.png
line 0 presidctions is: 
['385']
line 1 presidctions is: 
['385']
line 2 presidctions is: 
['385']
line 3 presidctions is: 
['385']
line 4 presidctions is: 
['385']
line 5 presidctions is: 
['385']
line 6 presidctions is: 
['385']
line 7 presidctions is: 
['385']
line 8 presidctions is: 
['385']
Prediction: 385
Time taken: 15.58
Test:  34
117
testCases\34\117\b05-038.png
testCases\34\117\b05-042.png
342
testCases\34\342\g06-011k.png
testCases\34\342\g06-018k.png
56

Prediction: 288
Time taken: 13.3
Test:  48
158
testCases\48\158\c04-008.png
testCases\48\158\c04-013.png
61
testCases\48\61\a04-047.png
testCases\48\61\a04-050.png
634
testCases\48\634\p06-030.png
testCases\48\634\p06-042.png
test.png
line 0 presidctions is: 
['634']
line 1 presidctions is: 
['634']
line 2 presidctions is: 
['634']
line 3 presidctions is: 
['634']
line 4 presidctions is: 
['634']
line 5 presidctions is: 
['634']
line 6 presidctions is: 
['634']
line 7 presidctions is: 
['634']
line 8 presidctions is: 
['634']
line 9 presidctions is: 
['634']
line 10 presidctions is: 
['634']
Prediction: 634
Time taken: 14.62
Test:  49
292
testCases\49\292\f07-076a.png
testCases\49\292\f07-081a.png
544
testCases\49\544\m04-000.png
testCases\49\544\m04-007.png
670
testCases\49\670\r06-035.png
testCases\49\670\r06-041.png
test.png
line 0 presidctions is: 
['292']
line 1 presidctions is: 
['292']
line 2 presidctions is: 
['292']
line 3 presidctions is: 
['292']
line 4 presidctions is: 
['2

352
testCases\63\352\g07-022b.png
testCases\63\352\g07-026b.png
59
testCases\63\59\a04-015.png
testCases\63\59\a04-019.png
test.png
line 0 presidctions is: 
['59']
line 1 presidctions is: 
['133']
line 2 presidctions is: 
['352']
line 3 presidctions is: 
['133']
line 4 presidctions is: 
['133']
line 5 presidctions is: 
['352']
line 6 presidctions is: 
['133']
Wrong classified at TestCase  63
Prediction: 133
Time taken: 14.37
Test:  64
117
testCases\64\117\b05-038.png
testCases\64\117\b05-042.png
152
testCases\64\152\c03-000c.png
testCases\64\152\c03-003c.png
243
testCases\64\243\e04-119.png
testCases\64\243\e04-124.png
test.png
line 0 presidctions is: 
['243']
line 1 presidctions is: 
['243']
line 2 presidctions is: 
['243']
line 3 presidctions is: 
['243']
line 4 presidctions is: 
['243']
line 5 presidctions is: 
['243']
line 6 presidctions is: 
['243']
Prediction: 243
Time taken: 14.08
Test:  65
151
testCases\65\151\c03-000b.png
testCases\65\151\c03-003b.png
154
testCases\65\154\c03-

test.png
line 0 presidctions is: 
['208']
line 1 presidctions is: 
['208']
line 2 presidctions is: 
['208']
line 3 presidctions is: 
['208']
line 4 presidctions is: 
['208']
line 5 presidctions is: 
['208']
line 6 presidctions is: 
['208']
line 7 presidctions is: 
['208']
line 8 presidctions is: 
['208']
line 9 presidctions is: 
['208']
line 10 presidctions is: 
['208']
Prediction: 208
Time taken: 17.97
Test:  79
125
testCases\79\125\b06-008.png
testCases\79\125\b06-056.png
204
testCases\79\204\d06-011.png
testCases\79\204\d06-107.png
209
testCases\79\209\d07-082.png
testCases\79\209\d07-085.png
test.png
line 0 presidctions is: 
['204']
line 1 presidctions is: 
['204']
line 2 presidctions is: 
['204']
line 3 presidctions is: 
['204']
line 4 presidctions is: 
['204']
line 5 presidctions is: 
['204']
line 6 presidctions is: 
['204']
line 7 presidctions is: 
['204']
line 8 presidctions is: 
['204']
line 9 presidctions is: 
['204']
line 10 presidctions is: 
['204']
Prediction: 204
Time tak

line 5 presidctions is: 
['287']
line 6 presidctions is: 
['287']
line 7 presidctions is: 
['287']
line 8 presidctions is: 
['287']
line 9 presidctions is: 
['287']
line 10 presidctions is: 
['287']
Prediction: 287
Time taken: 18.51
Test:  94
285
testCases\94\285\f07-000.png
testCases\94\285\f07-039a.png
300
testCases\94\300\g01-067.png
testCases\94\300\g01-070.png
89
testCases\94\89\b01-027.png
testCases\94\89\b01-033.png
test.png
line 0 presidctions is: 
['285']
line 1 presidctions is: 
['285']
line 2 presidctions is: 
['285']
line 3 presidctions is: 
['285']
line 4 presidctions is: 
['285']
line 5 presidctions is: 
['285']
line 6 presidctions is: 
['285']
line 7 presidctions is: 
['285']
Prediction: 285
Time taken: 14.87
Test:  95
207
testCases\95\207\d06-037.png
testCases\95\207\d06-072.png
339
testCases\95\339\g06-011h.png
testCases\95\339\g06-018h.png
93
testCases\95\93\b01-113.png
testCases\95\93\b01-118.png
test.png
line 0 presidctions is: 
['339']
line 1 presidctions is: 
['33